In [11]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from itertools import combinations 
from numpy import savetxt
import random

In [12]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import PolynomialFeatures
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, Input
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras

In [30]:
input_df = pd.read_csv("data/input_data_nn.csv", sep=";")
output_df = pd.read_csv("data/output_data_nn.csv", sep=";")
df = pd.concat([input_df, output_df], axis=1)
df = df.loc[~(df==0).all(axis=1)].reset_index()

In [24]:
output_df.describe()

,tensao_barramento11,tensao_barramento12,tensao_barramento24,pot_ativa_inj_barramento11,pot_ativa_inj_barramento12,pot_ativa_inj_barramento24,pot_reativa_inj_barramento11,pot_reativa_inj_barramento12,pot_reativa_inj_barramento24
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,0.996569,0.997564,0.993244,44.794038,65.017462,98.531922,-15.602551,-3.119726,25.755603
std,0.021639,0.021008,0.021868,32.318890,46.834371,53.707492,11.060453,15.983727,10.808798
min,0.952364,0.953941,0.950000,-27.531718,-45.061452,-29.154621,-34.601205,-30.758948,-0.197103
25%,0.986579,0.989016,0.981990,17.824650,35.544467,54.846269,-23.606006,-15.992032,19.179831
50%,1.005131,1.006796,1.001604,43.524182,61.976083,95.024840,-17.984876,-9.813467,26.085991
75%,1.012191,1.012873,1.011476,66.443556,94.266833,134.378609,-7.401176,8.937662,33.428328
max,1.034885,1.032754,1.022086,159.904392,233.978721,261.648979,16.029554,47.883536,57.932657


In [25]:
output_df.describe()

,tensao_barramento11,tensao_barramento12,tensao_barramento24,pot_ativa_inj_barramento11,pot_ativa_inj_barramento12,pot_ativa_inj_barramento24,pot_reativa_inj_barramento11,pot_reativa_inj_barramento12,pot_reativa_inj_barramento24
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,0.996569,0.997564,0.993244,44.794038,65.017462,98.531922,-15.602551,-3.119726,25.755603
std,0.021639,0.021008,0.021868,32.318890,46.834371,53.707492,11.060453,15.983727,10.808798
min,0.952364,0.953941,0.950000,-27.531718,-45.061452,-29.154621,-34.601205,-30.758948,-0.197103
25%,0.986579,0.989016,0.981990,17.824650,35.544467,54.846269,-23.606006,-15.992032,19.179831
50%,1.005131,1.006796,1.001604,43.524182,61.976083,95.024840,-17.984876,-9.813467,26.085991
75%,1.012191,1.012873,1.011476,66.443556,94.266833,134.378609,-7.401176,8.937662,33.428328
max,1.034885,1.032754,1.022086,159.904392,233.978721,261.648979,16.029554,47.883536,57.932657


In [26]:
def augment(x, y, augmenter = None):
    x, augmenter = _add_interactions(x, augmenter)
    return x, y, augmenter, x.columns

def _add_interactions(df, augmenter):
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns) + ['_'.join(x) for x in combos]

    if augmenter:
        df = augmenter.transform(df)
    else:
        augmenter = PolynomialFeatures(interaction_only=True, include_bias=False)
        df = augmenter.fit_transform(df)

    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indicies = [i for i, x in enumerate(list((df == 0).all())) if x]
    df = df.drop(df.columns[noint_indicies], axis=1)
    
    return df, augmenter

In [31]:
def run_main(df, idx, idx2, ids_to_pred):    
    cols = [
        'carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv',
        'carga_subrede_230kv',
        'pot_eolica_subrede_230kv',
        'pot_solar_subrede_230kv'
    ]
    
    cols_output = [
        'tensao_barramento11', 'tensao_barramento12', 'tensao_barramento24',
        'pot_ativa_inj_barramento11', 'pot_ativa_inj_barramento12', 'pot_ativa_inj_barramento24',
        'pot_reativa_inj_barramento11', 'pot_reativa_inj_barramento12', 'pot_reativa_inj_barramento24'
    ]
    predicted_df = pd.DataFrame(columns=cols_output, index=ids_to_pred)
    new_df = df.copy()

    cols_pred = ['tensao_barramento11']
    model_id = 'tensao_11'
    tensao_11_df, predicted_11 = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred, update_test=True)

    cols_pred = ['tensao_barramento12']
    model_id = 'tensao_12'
    tensao_12_df, predicted_12 = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred, update_test=True)

    cols_pred = ['tensao_barramento24']
    model_id = 'tensao_24'
    tensao_24_df, predicted_24 = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred, update_test=True)
    
    new_df['tensao_barramento11'] = tensao_11_df['tensao_barramento11']
    new_df['tensao_barramento12'] = tensao_12_df['tensao_barramento12']
    new_df['tensao_barramento24'] = tensao_24_df['tensao_barramento24']
    
    predicted_df['tensao_barramento11'] = predicted_11
    predicted_df['tensao_barramento12'] = predicted_12
    predicted_df['tensao_barramento24'] = predicted_24
    
    cols += ['tensao_barramento11', 'tensao_barramento12', 'tensao_barramento24']
    
    cols_pred = ['pot_ativa_inj_barramento11']
    model_id = 'pot_ativa_inj_barramento11'
    result_df, predictions = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred)
    predicted_df[model_id] = predictions

    cols_pred = ['pot_ativa_inj_barramento12']
    model_id = 'pot_ativa_inj_barramento12'
    result_df, predictions = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred)
    predicted_df[model_id] = predictions

    cols_pred = ['pot_ativa_inj_barramento24']
    model_id = 'pot_ativa_inj_barramento24'
    result_df, predictions = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred)
    predicted_df[model_id] = predictions
    
    cols_pred = ['pot_reativa_inj_barramento11']
    model_id = 'pot_reativa_inj_barramento11'
    result_df, predictions = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred)
    predicted_df[model_id] = predictions
    
    cols_pred = ['pot_reativa_inj_barramento12']
    model_id = 'pot_reativa_inj_barramento12'
    result_df, predictions = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred)
    predicted_df[model_id] = predictions

    cols_pred = ['pot_reativa_inj_barramento24']
    model_id = 'pot_reativa_inj_barramento24'
    result_df, predictions = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred)
    predicted_df[model_id] = predictions    
    
    predicted_df.to_csv(f'{idx}-{idx2}.csv')

    return new_df

In [32]:
def run(df, cols, cols_pred, model_id, idx, ids_to_pred, update_test = False):    
    _df = df[cols + cols_pred]
    
    X, Y = _df[cols], _df[cols_pred]
    X, Y, _, x_cols, = augment(X, Y)
    _df_augm = pd.concat([X, Y],  axis=1)

    df_for_testing = _df_augm.loc[ids_to_pred]
    df_for_training = _df_augm.loc[~_df_augm.index.isin(ids_to_pred)]
    
    df_for_training = df_for_training.sample(frac=1)
    
    trainX, trainY = df_for_training.drop(cols_pred, axis=1), df_for_training[cols_pred]
    testX, testY = df_for_testing.drop(cols_pred, axis=1), df_for_testing[cols_pred]
    
    scaler = StandardScaler()
    trainX = scaler.fit_transform(trainX)
    testX = scaler.transform(testX)
    
    print(f" ========== MODEL {model_id} {idx} ============")
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)
    mc = ModelCheckpoint(f"models-new/ann-{model_id}-{idx}.h5", monitor='val_loss', mode='min', save_best_only=True, verbose=0)

    model = Sequential()
    model.add(Input(shape=(trainX.shape[1],)))
    model.add(Dense(trainX.shape[-1] ** 2, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(trainX.shape[-1], activation = 'relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(len(cols_pred)))

    model.compile(loss = 'mae', optimizer = 'adam')
    model.fit(
        trainX, trainY,
        batch_size = 10, epochs = 20,
        callbacks = [es, mc],
        validation_data = (testX, testY),
        verbose=0
    )

    _predictions = model.predict(testX)    
    rmse = sqrt(mean_squared_error(testY, _predictions.flatten()))
    print(f"RMSE: {rmse}")
        
    results.append(rmse)
    
    if update_test:
        df_for_testing[cols_pred] = _predictions

    return pd.concat([df_for_training, df_for_testing]), _predictions  

In [29]:
indexes = np.array(df.index.tolist())
random.shuffle(indexes)
ids_to_pred = np.split(indexes, 3)

results_all = []
for i in range(1):
    results_per_ids = []
    for idx2 in range(len(ids_to_pred)):
        results = []
        run_main(df, i, idx2, ids_to_pred[idx2])
        results_per_ids.append(results)
        
    results_all.append(results_per_ids)

 ========== MODEL tensao_11 0 ============
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When 

KeyboardInterrupt: 

In [21]:
for _result in results_all:
    print(results_all.index(_result))
    for fold in _result:
        print(f"{_result.index(fold)}: {sum(fold)}")

0
0: 32.33497086128333
1: 36.889005154036376
2: 36.68749314981343
3: 31.932388827029182
1
0: 32.26412368037137
1: 31.990872924834257
2: 36.0298650015969
3: 32.30431549681663
2
0: 31.72844917901247
1: 29.84034210963254
2: 34.37873809844469
3: 38.184718100379435
3
0: 34.15181711482428
1: 30.558933149246954
2: 30.1250874789621
3: 31.1846092629998
4
0: 34.86386086977219
1: 28.93438134306144
2: 36.44611309999486
3: 33.4754096497843
5
0: 37.116041669193386
1: 37.296140359486756
2: 30.919280629845613
3: 39.44656374003831
6
0: 31.38229179727558
1: 30.653259296110438
2: 31.444674673125057
3: 36.76260480496647
7
0: 28.349547201857266
1: 33.73417886152881
2: 39.737148167795624
3: 37.59890026817541
8
0: 29.559890665146096
1: 40.83094120187875
2: 26.866222288396877
3: 37.813916555230335
9
0: 38.21746676037965
1: 37.003628372655385
2: 47.06004230959253
3: 34.13258223262406


In [26]:
print(f"Fold 0: {results_all[7][0]}")
print(f"Fold 1: {results_all[4][1]}")
print(f"Fold 2: {results_all[8][2]}")
print(f"Fold 3: {results_all[3][3]}")

Fold 0: [0.005618806104382909, 0.006374496128408189, 0.005003273984089787, 3.4735610445018055, 5.057355698378199, 3.6107973146928116, 7.466659649968084, 6.293739685672916, 2.4304372324265673]
Fold 1: [0.004912833436191945, 0.006315874222206748, 0.0050400543592956865, 5.882427740243566, 3.5072674248462787, 3.786373119903367, 6.751693446504084, 6.168161926473203, 2.822188923073245]
Fold 2: [0.004500589408310584, 0.004620877592182861, 0.005015842571451259, 3.953451113791048, 4.887246335988537, 4.617532544758241, 5.595597117037946, 5.241330146546544, 2.5569277207026158]
Fold 3: [0.005904257261356303, 0.004717165985727671, 0.005687405161361025, 3.2969980512393264, 5.985023419883928, 3.5217441168282497, 8.310837484354026, 6.692027891640731, 3.3616694706450936]


In [32]:
df_7_0 = pd.read_csv('7-0.csv')
df_7_0 = df_7_0.rename(columns={"Unnamed: 0": "id"})

df_4_1 = pd.read_csv('4-1.csv')
df_4_1 = df_4_1.rename(columns={"Unnamed: 0": "id"})

df_8_2 = pd.read_csv('8-2.csv')
df_8_2 = df_8_2.rename(columns={"Unnamed: 0": "id"})

df_3_3 = pd.read_csv('3-3.csv')
df_3_3 = df_3_3.rename(columns={"Unnamed: 0": "id"})

In [36]:
df_best_predicted = pd.concat([df_7_0, df_4_1, df_8_2, df_3_3])
df_best_predicted = df_best_predicted.sort_values(by=['id'])

In [41]:
df_best_predicted.columns

Index(['id', 'tensao_barramento11', 'tensao_barramento12',
       'tensao_barramento24', 'pot_ativa_inj_barramento11',
       'pot_ativa_inj_barramento12', 'pot_ativa_inj_barramento24',
       'pot_reativa_inj_barramento11', 'pot_reativa_inj_barramento12',
       'pot_reativa_inj_barramento24'],
      dtype='object')